# ALE

This notebook will guide you through the process of setting up and running an ALE meta-analysis.

First we need to set up two excel files that will serve as input for all following steps:

#### 1. Experiment information

Please use the following format / syntax:

- The spreadsheet should have a title row that will not be read
- The order of the columns should be as in this example
- Tags are optional
- At the moment numeric tags are not working!

![title](img/exp_info.png)

**Important note: In either case the tags are taken straight as they are, so please make sure that spelling (including spaces etc) is consistent throughout the file!**


#### 2. Analysis information

Please use the following format / syntax:

![title](img/meta_info.png)

**1st column**: 
- M denotes main effect
- P denotes probabilistic / subsampled / CV'd ALE
    - Need to specify the desired N after the P, e.g. P17
- C denotes contrast (this line and the following are contrasted against each other). If the respective main effects in the two lines [e.g. 3 and 4] are not yet computed, they will be evaluated
- B denotes balanced contrast. This way both analysis will be based on the same N. (Option to specify the desired N after the B, e.g. B17)
    - Option to specify the desired N after the B, e.g. B17. If no N is provided N will be equal to the smaller analysis N - 2 or the mean between the smaller analysis N and 17 depending on which is smaller.

**2nd column**:
Titel of the effect. Ideally avoid spaces and non file-name characters. When (re-) entering a title as part of a contrast, please keep naming/spelling consistent otherwise the “old” main effect is not recognized

**3rd column onwards: Specify the analysis by tags.**
- +ALL   ->   Include the entire set of experiments in that matfile
- +Tag   ->    Include only those experiments that have the respective label
- +Tag1  +Tag2   -> Conjunction, includes only experiments that have both labels
- +Tag1  +Tag2   ?   ->  Logical ‘OR’, includes experiments that have either label
- -Tag   ->   Exclude experiments that have the respective label
- $VOI   ->  additionally to the voxel-wise analysis perform VOI analysis for the specified VOI [full path/filename, binary mask]. Multiple VOIs can be entered after each other and are evaluated at the same time. Furthermore an atlas can be added and a ROI analysis will be run for each distinct region specified in the atlas

## How to run your analyses

1. Create a folder in which you would like to run your analysis (can be the folder you cloned this repository to)
2. Copy both excel sheets into the folder
3. Enter the path to your folder and the names of your two excel sheets into the first code cell below
4. Run your analysis by running all the code cells below. There will be output printed below indicating your process.
 
**Attention: Depending on your knowledge level you can either run the basic routine taking over all default parameter values or run the advanced routine, where you can change the parameter settings**

In [ ]:
path = "absolute/path/to/your/folder"
analysis_info_name = "analysis_info_file.xlsx"
experiment_info_name = "experiment_info_file.xlsx"

In [ ]:
from nb_pipeline import setup, analysis
meta_df, exp_all, tasks = setup(path, analysis_info_name, experiment_info_name)

In [ ]:
"""Basic routine"""
analysis(path, meta_df, exp_all, tasks)

In [ ]:
### """Advanced Routine"""

# if TFCE should be included in the main effect analysis. very computationally expensive and does not give big benefits over cFWE.
tfce_enabled=False

# number of processes spawned when parallelizing. ideally should not exceed the amount of cores of the machine.
nprocesses=4

# amount of reptitions used for null distribution simulation (noise samples). Convergence starts at around 1000, recommended value = 10000
# Used in all types of analysis
null_repeats=5000

# significance threshold used for cluster forming. recommended value = 0.001
# Used in full MainEffect analysis
cluster_thresh=0.001

# Amount of samples taken from study population, recommended value = 2500
# Used in probabilistic and balanced contrast
sample_n=2500

# Threshold used to compare differences against. recommended value = 0.05
# Used in ("legacy") contrast
diff_thresh=0.05

# Whether or not the contrast is only calculated in areas, which are significant in the main_effect analysis
# Used in ("legacy") contrast
masking=False

# Amount of sampling repititions used when comparing two subsampled ALEs. recommended value > 500
# Used in balanced contrast
# Note: As this difference permutation is performed inside of the null distribution simulation it gets repeated X (null_repeats) amount of times
# - this can lead to a small increase in diff_repeats leading to a big increase in computation time
diff_repeats=500

analysis(path = path,
         meta_df = meta_df,
         exp_all = exp_all,
         tasks = tasks,
         tfce_enabled = tfce_enabled,
         null_repeats = null_repeats,
         cluster_thresh = cluster_thresh,
         sample_n = sample_n,
         diff_thresh = diff_thresh,
         masking = masking,
         diff_repeats = diff_repeats,
         nprocesses = nprocesses)